### Imports


In [1]:
import os
import json
import datetime
from envyaml import EnvYAML
import pandas as pd
from tsfresh import extract_features


### Accessing datasets


In [3]:
def read_csv(path):
    return pd.read_csv(path)


def to_csv(df, path):
    df.to_csv(path, index=False)


E4 dataset directories


In [4]:
VAR_ENV = EnvYAML("../../../env.yaml")
path_list = [os.path.join("../../..", VAR_ENV["dataset.path"], VAR_ENV["dataset.version"],
                          f"P{i:02d}", "E4") for i in range(1, VAR_ENV["dataset.participants"] + 1)]

### Dataframes


In [29]:
study_information = pd.read_csv(os.path.join("../../..", VAR_ENV["dataset.path"], VAR_ENV["dataset.version"],"Study_Information.csv"))
activities = ["Start_Sit", "Start_Stand", "Start_Cycle1", "Start_Cycle2", "Start_Run1", "Start_Run2"]
time_windows = [2,4,6,8,10,12]
for time_window in time_windows[5:]:
    number_of_rows = 32 * time_window
    for path in path_list[:1]:
        # dataset manipulation
        df_temp = pd.read_csv(os.path.join(
            path, 'modified_0_rows_removed_ACC.csv'))
        df_temp["id"] = path_list.index(path)+1

        dictionary = {
            'session_type': [],
            'window_number': [],
            'start_timestamp': [],
            'end_timestamp': [],
            'ax_sum_values_wristband_wrist_acc': [],
            'ax_median_wristband_wrist_acc': [],
            'ax_mean_wristband_wrist_acc': [],
            'ax_length_wristband_wrist_acc': [],
            'ax_standard_deviation_wristband_wrist_acc': [],
            'ax_variance_wristband_wrist_acc': [],
            'ax_root_mean_square_wristband_wrist_acc': [],
            'ax_maximum_wristband_wrist_acc': [],
            'ax_absolute_maximum_wristband_wrist_acc': [],
            'ax_minimum_wristband_wrist_acc': [],
            'ay_sum_values_wristband_wrist_acc': [],
            'ay_median_wristband_wrist_acc': [],
            'ay_mean_wristband_wrist_acc': [],
            'ay_length_wristband_wrist_acc': [],
            'ay_standard_deviation_wristband_wrist_acc': [],
            'ay_variance_wristband_wrist_acc': [],
            'ay_root_mean_square_wristband_wrist_acc': [],
            'ay_maximum_wristband_wrist_acc': [],
            'ay_absolute_maximum_wristband_wrist_acc': [],
            'ay_minimum_wristband_wrist_acc': [],
            'az_sum_values_wristband_wrist_acc': [],
            'az_median_wristband_wrist_acc': [],
            'az_mean_wristband_wrist_acc': [],
            'az_length_wristband_wrist_acc': [],
            'az_standard_deviation_wristband_wrist_acc': [],
            'az_variance_wristband_wrist_acc': [],
            'az_root_mean_square_wristband_wrist_acc': [],
            'az_maximum_wristband_wrist_acc': [],
            'az_absolute_maximum_wristband_wrist_acc': [],
            'az_minimum_wristband_wrist_acc': [],
        }
        current_activity_index = 0
        current_window = -1
        i = 0
        while i < len(df_temp)-number_of_rows:
            i += number_of_rows
            start = df_temp['timestamp'][i]
            end = df_temp['timestamp'][i+number_of_rows-1]
            extracted_features = extract_features(df_temp[i:i+number_of_rows], column_id="id", column_sort="timestamp", default_fc_parameters={
                "sum_values": None,
                "mean": None,
                "median": None,
                "length": None,
                "standard_deviation": None,
                "variance": None,
                "root_mean_square": None,
                "maximum": None,
                "absolute_maximum": None,
                "minimum": None,
            })
            row_end_time = datetime.datetime.fromisoformat(end.split('.')[0])
            if(current_activity_index >= 5):
                next_activity_start_time = datetime.datetime.fromisoformat(df_temp.loc[df_temp.index[-1], 'timestamp'])
            else:
                next_activity_start_time = datetime.datetime.fromisoformat(study_information[activities[current_activity_index+1]][path_list.index(path)])
            if(row_end_time+datetime.timedelta(seconds=12) > next_activity_start_time):
                i = str(next_activity_start_time)
                current_activity_index += 1
                current_window = 0
                continue
            if (row_end_time < next_activity_start_time):
                # df.loc[i,'activity'] = current_activity_index
                dictionary['session_type'].append(current_activity_index)
                current_window += 1
                dictionary['window_number'].append(current_window)
            elif (current_activity_index == 5):
                dictionary['session_type'].append(5)
                current_window += 1
                dictionary['window_number'].append(current_window)
            else:
                current_activity_index += 1
                # df.loc[i,'activity'] = current_activity_index
                dictionary['session_type'].append(current_activity_index)
                current_window = 0
                dictionary['window_number'].append(current_window)
            dictionary['start_timestamp'].append(start)
            dictionary['end_timestamp'].append(end)
            dictionary['ax_sum_values_wristband_wrist_acc'].append(
                extracted_features["acc_x_avg__sum_values"][path_list.index(path)+1])
            dictionary['ax_median_wristband_wrist_acc'].append(
                extracted_features["acc_x_avg__median"][path_list.index(path)+1])
            dictionary['ax_mean_wristband_wrist_acc'].append(
                extracted_features["acc_x_avg__mean"][path_list.index(path)+1])
            dictionary['ax_length_wristband_wrist_acc'].append(
                extracted_features["acc_x_avg__length"][path_list.index(path)+1])
            dictionary['ax_standard_deviation_wristband_wrist_acc'].append(
                extracted_features["acc_x_avg__standard_deviation"][path_list.index(path)+1])
            dictionary['ax_variance_wristband_wrist_acc'].append(
                extracted_features["acc_x_avg__variance"][path_list.index(path)+1])
            dictionary['ax_root_mean_square_wristband_wrist_acc'].append(
                extracted_features["acc_x_avg__root_mean_square"][path_list.index(path)+1])
            dictionary['ax_maximum_wristband_wrist_acc'].append(
                extracted_features["acc_x_avg__maximum"][path_list.index(path)+1])
            dictionary['ax_absolute_maximum_wristband_wrist_acc'].append(
                extracted_features["acc_x_avg__absolute_maximum"][path_list.index(path)+1])
            dictionary['ax_minimum_wristband_wrist_acc'].append(
                extracted_features["acc_x_avg__minimum"][path_list.index(path)+1])
            dictionary['ay_sum_values_wristband_wrist_acc'].append(
                extracted_features["acc_y_avg__sum_values"][path_list.index(path)+1])
            dictionary['ay_median_wristband_wrist_acc'].append(
                extracted_features["acc_y_avg__median"][path_list.index(path)+1])
            dictionary['ay_mean_wristband_wrist_acc'].append(
                extracted_features["acc_y_avg__mean"][path_list.index(path)+1])
            dictionary['ay_length_wristband_wrist_acc'].append(
                extracted_features["acc_y_avg__length"][path_list.index(path)+1])
            dictionary['ay_standard_deviation_wristband_wrist_acc'].append(
                extracted_features["acc_y_avg__standard_deviation"][path_list.index(path)+1])
            dictionary['ay_variance_wristband_wrist_acc'].append(
                extracted_features["acc_y_avg__variance"][path_list.index(path)+1])
            dictionary['ay_root_mean_square_wristband_wrist_acc'].append(
                extracted_features["acc_y_avg__root_mean_square"][path_list.index(path)+1])
            dictionary['ay_maximum_wristband_wrist_acc'].append(
                extracted_features["acc_y_avg__maximum"][path_list.index(path)+1])
            dictionary['ay_absolute_maximum_wristband_wrist_acc'].append(
                extracted_features["acc_y_avg__absolute_maximum"][path_list.index(path)+1])
            dictionary['ay_minimum_wristband_wrist_acc'].append(
                extracted_features["acc_y_avg__minimum"][path_list.index(path)+1])
            dictionary['az_sum_values_wristband_wrist_acc'].append(
                extracted_features["acc_z_avg__sum_values"][path_list.index(path)+1])
            dictionary['az_median_wristband_wrist_acc'].append(
                extracted_features["acc_z_avg__median"][path_list.index(path)+1])
            dictionary['az_mean_wristband_wrist_acc'].append(
                extracted_features["acc_z_avg__mean"][path_list.index(path)+1])
            dictionary['az_length_wristband_wrist_acc'].append(
                extracted_features["acc_z_avg__length"][path_list.index(path)+1])
            dictionary['az_standard_deviation_wristband_wrist_acc'].append(
                extracted_features["acc_z_avg__standard_deviation"][path_list.index(path)+1])
            dictionary['az_variance_wristband_wrist_acc'].append(
                extracted_features["acc_z_avg__variance"][path_list.index(path)+1])
            dictionary['az_root_mean_square_wristband_wrist_acc'].append(
                extracted_features["acc_z_avg__root_mean_square"][path_list.index(path)+1])
            dictionary['az_maximum_wristband_wrist_acc'].append(
                extracted_features["acc_z_avg__maximum"][path_list.index(path)+1])
            dictionary['az_absolute_maximum_wristband_wrist_acc'].append(
                extracted_features["acc_z_avg__absolute_maximum"][path_list.index(path)+1])
            dictionary['az_minimum_wristband_wrist_acc'].append(
                extracted_features["acc_z_avg__minimum"][path_list.index(path)+1])

        df = pd.DataFrame(dictionary)
        user_id = [path_list.index(path)+1 for i in range(len(df))]
        df.insert(0, 'user_id', user_id, True)

        # saving
        to_csv(df, os.path.join("../../..",
            VAR_ENV["dataset.path"], VAR_ENV["dataset.version"], f"P{path_list.index(path)+1:02d}", "E4", f"modified_with_tsfresh_ACC_{time_window}.csv"))


Feature Extraction: 100%|██████████| 3/3 [00:00<00:00, 899.29it/s]


TypeError: '<' not supported between instances of 'str' and 'int'